<a href="https://colab.research.google.com/github/2003UJAN/SIH_RE-DACT/blob/main/RE_DACT_Individual_Text%26Image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import spacy
import cv2
import os
from sklearn.metrics import precision_score, recall_score, f1_score

In [16]:
nlp = spacy.load("en_core_web_sm")

In [17]:
def perform_ner(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

In [18]:
def redact_text(text, entities, degree):
    for entity, label in entities:
        if degree == 1:
            if label in ["PERSON", "ORG"]:
                text = text.replace(entity, "[REDACTED]")
        elif degree == 2:
            if label in ["PERSON", "ORG", "DATE"]:
                text = text.replace(entity, "[REDACTED]")
        elif degree == 3:
            text = text.replace(entity, "[REDACTED]")
    return text

In [19]:
def save_text_file(content, filename="redacted_text.txt"):
    with open(filename, 'w') as file:
        file.write(content)
    print(f"Redacted text saved as {filename}")

In [20]:
def handle_text_redaction_own_input():
    input_text = input("Enter text to redact: ")
    degree = int(input("Enter redaction degree (1-3): "))
    entities = perform_ner(input_text)
    redacted_text = redact_text(input_text, entities, degree)
    save_text_file(redacted_text)

In [21]:
def redact_image_with_haar(image_path, degree=1):
    image = cv2.imread(image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        if degree == 1:
            face = image[y:y+h, x:x+w]
            blurred_face = cv2.GaussianBlur(face, (99, 99), 30)
            image[y+y+h, x:x+w] = blurred_face
        elif degree == 2:
            face = image[y:y+h, x:x+w]
            small_face = cv2.resize(face, (w // 10, h // 10), interpolation=cv2.INTER_LINEAR)
            resized_face = cv2.resize(small_face, (w, h), interpolation=cv2.INTER_NEAREST)
            image[y:y+h, x:x+w] = resized_face
        elif degree == 3:
            image[y:y+h, x:x+w] = 0

    return image

In [22]:
def save_image_file(image, filename="redacted_image.jpg"):
    cv2.imwrite(filename, image)
    print(f"Redacted image saved as {filename}")

In [23]:
def handle_image_redaction_own_input():
    image_path = input("Enter image file path: ")
    degree = int(input("Enter redaction degree (1-3): "))

    redacted_image = redact_image_with_haar(image_path, degree)

    save_image_file(redacted_image)

In [24]:
def handle_text_redaction_from_dataset(dataset_dir):
    for filename in os.listdir(dataset_dir):
        if filename.endswith(".txt"):
            file_path = os.path.join(dataset_dir, filename)
            with open(file_path, 'r') as file:
                text = file.read()
            degree = int(input(f"Enter redaction degree for {filename} (1-3): "))
            entities = perform_ner(text)
            redacted_text = redact_text(text, entities, degree)
            save_text_file(redacted_text, filename=f"redacted_{filename}")

In [25]:
def handle_image_redaction_from_dataset(dataset_dir):
    for filename in os.listdir(dataset_dir):
        if filename.endswith((".jpg", ".jpeg", ".png")):
            file_path = os.path.join(dataset_dir, filename)
            degree = int(input(f"Enter redaction degree for {filename} (1-3): "))
            redacted_image = redact_image_with_haar(file_path, degree)
            save_image_file(redacted_image, filename=f"redacted_{filename}")

In [28]:
def main():
    print("1. Redact Own Input Text")
    print("2. Redact Own Input Image")
    print("3. Redact Text from Dataset")
    print("4. Redact Image from Dataset")
    choice = int(input("Enter your choice: "))

    if choice == 1:
        handle_text_redaction_own_input()
    elif choice == 2:
        handle_image_redaction_own_input()
    elif choice == 3:
        dataset_dir = input("Enter the dataset directory path: ")
        handle_text_redaction_from_dataset(dataset_dir)
    elif choice == 4:
        dataset_dir = input("Enter the dataset directory path: ")
        handle_image_redaction_from_dataset(dataset_dir)

if __name__ == "__main__":
    main()

1. Redact Own Input Text
2. Redact Own Input Image
3. Redact Text from Dataset
4. Redact Image from Dataset
Enter your choice: 2
Enter image file path: /content/Pankaj-Sivdutt-Dual.jpeg
Enter redaction degree (1-3): 3
Redacted image saved as redacted_image.jpg
